# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [ ]:
import cv2 as cv
import numpy as np
import os, os.path
import xml.etree.ElementTree as ET

### Useful functions

In [ ]:
# Convert image color model to hsv
def cvt_color_to_hsv(img):
    cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Remove image noise with gaussian blur (better at preserving edges)
def remove_noise(img):
    return cv.GaussianBlur(img, 3)

# Add contrast to image by histogram equalization
def add_contrast(img):
    return cv.equalizeHist(img)

# Filter reds in image using a mask by converting colors different than red black
def filter_red(img):
    # lower mask (0-10)
    lower_red = np.array([0,70,50])
    upper_red = np.array([10,255,255])
    mask0 = cv.inRange(img, lower_red, upper_red)  

    # upper mask (170-180)
    lower_red = np.array([170,70,50])
    upper_red = np.array([180,255,255])
    mask1 = cv.inRange(img, lower_red, upper_red)

    mask = mask0 | mask1

    img_aux = img.copy()

    img_aux[np.where(mask == 0)] = 0

    return img_aux

# Filter blues in image using a mask by converting colors different than blue black
def filter_blue(img):
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])
    mask = cv.inRange(img, lower_blue, upper_blue)  

    img_aux = img.copy()

    img_aux[np.where(mask == 0)] = 0

    return img_aux

To start we need to collect all of the xml files and the corresponding png files so we can parse them.
Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [5]:
xml_files = [name for name in os.listdir('./Dataset/annotations')]
validation_dictionary = {}

for name in xml_files:
    # Parse the xml
    mytree = ET.parse('./Dataset/annotations/' + name)
    
    # Get number from name
    order = ''.join(i for i in name if i.isdigit())
    
    # Get sign type
    validation_dictionary[order] = mytree.getroot().find('object').find('name').text
    

Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [6]:
sign_types = []
for entry in validation_dictionary:
    if validation_dictionary[entry] not in sign_types:
        sign_types.append(validation_dictionary[entry])
print(sign_types)
print(validation_dictionary)

['trafficlight', 'speedlimit', 'crosswalk', 'stop']
{'0': 'trafficlight', '1': 'trafficlight', '10': 'trafficlight', '100': 'speedlimit', '101': 'speedlimit', '102': 'speedlimit', '103': 'speedlimit', '104': 'speedlimit', '105': 'speedlimit', '106': 'speedlimit', '107': 'speedlimit', '108': 'speedlimit', '109': 'speedlimit', '11': 'trafficlight', '110': 'speedlimit', '111': 'speedlimit', '112': 'speedlimit', '113': 'speedlimit', '114': 'speedlimit', '115': 'speedlimit', '116': 'speedlimit', '117': 'speedlimit', '118': 'speedlimit', '119': 'speedlimit', '12': 'trafficlight', '120': 'speedlimit', '121': 'speedlimit', '122': 'crosswalk', '123': 'crosswalk', '124': 'crosswalk', '125': 'crosswalk', '126': 'crosswalk', '127': 'crosswalk', '128': 'crosswalk', '129': 'crosswalk', '13': 'trafficlight', '130': 'crosswalk', '131': 'crosswalk', '132': 'crosswalk', '133': 'crosswalk', '134': 'crosswalk', '135': 'crosswalk', '136': 'crosswalk', '137': 'crosswalk', '138': 'crosswalk', '139': 'crosswa

## Stop signs

RuntimeError: dictionary changed size during iteration